# 주차장 전처리

In [2]:
import numpy as np
import pandas as pd
pd.set_option("display.max_rows", 100, "display.max_columns", 100)

import warnings
warnings.filterwarnings('ignore')

import re
from tqdm import tqdm

import requests
import json

In [3]:
# PATH = 'G:/다른 컴퓨터/My_desktop/project/데이터 분석 프로젝트/이노포스트/data/'
# PATH = 'D:/project/데이터 분석 프로젝트/이노포스트/data/'
PATH = 'G:/다른 컴퓨터/My_desktop/project/데이터 분석 프로젝트/이노포스트/data/'

In [4]:
df = pd.read_csv(PATH + '06_주차장/전국주차장정보표준데이터.csv', encoding='cp949')

In [5]:
df.shape

(15315, 33)

In [6]:
df.head(2)

,주차장관리번호,주차장명,주차장구분,주차장유형,소재지도로명주소,소재지지번주소,주차구획수,급지구분,부제시행구분,운영요일,평일운영시작시각,평일운영종료시각,토요일운영시작시각,토요일운영종료시각,공휴일운영시작시각,공휴일운영종료시각,요금정보,주차기본시간,주차기본요금,추가단위시간,추가단위요금,1일주차권요금적용시간,1일주차권요금,월정기권요금,결제방법,특기사항,관리기관명,전화번호,위도,경도,데이터기준일자,제공기관코드,제공기관명
0,156-2-000043,송현주공시장 공영 주차장,공영,노외,대구광역시 달서구 송현로8안길 6,대구광역시 달서구 송현동 1949-21,26,2,미시행,평일+토요일+공휴일,09:00,22:00,09:00,22:00,09:00,22:00,유료,30.0,300,10,200,4.5,5000.0,60000.0,현금,"정상금액 기준: 경차60퍼센트할인, 장애인2시간면제 50퍼센트할인, 친환경적, 저공...",대구광역시 달서구청,053-667-2656,35.822270,128.548959,2021-09-13,3470000,대구광역시 달서구
1,156-2-000044,송현2동마을 공영 주차장,공영,노외,대구광역시 달서구 월배로75길 112,대구광역시 달서구 송현동 496-1,42,3,미시행,평일+토요일+공휴일,00:00,00:00,00:00,00:00,00:00,00:00,무료,1440.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,대구광역시 달서구청,053-667-3092,35.832459,128.547577,2021-09-13,3470000,대구광역시 달서구


In [7]:
df.columns

Index(['주차장관리번호', '주차장명', '주차장구분', '주차장유형', '소재지도로명주소', '소재지지번주소', '주차구획수',
       '급지구분', '부제시행구분', '운영요일', '평일운영시작시각', '평일운영종료시각', '토요일운영시작시각',
       '토요일운영종료시각', '공휴일운영시작시각', '공휴일운영종료시각', '요금정보', '주차기본시간', '주차기본요금',
       '추가단위시간', '추가단위요금', '1일주차권요금적용시간', '1일주차권요금', '월정기권요금', '결제방법', '특기사항',
       '관리기관명', '전화번호', '위도', '경도', '데이터기준일자', '제공기관코드', '제공기관명'],
      dtype='object')

## 1차 컬럼 선별

In [8]:
df2 = df[['주차장명', '주차장유형', '소재지도로명주소', '소재지지번주소', '위도', '경도', '제공기관명']]
df2

,주차장명,주차장유형,소재지도로명주소,소재지지번주소,위도,경도,제공기관명
0,송현주공시장 공영 주차장,노외,대구광역시 달서구 송현로8안길 6,대구광역시 달서구 송현동 1949-21,35.822270,128.548959,대구광역시 달서구
1,송현2동마을 공영 주차장,노외,대구광역시 달서구 월배로75길 112,대구광역시 달서구 송현동 496-1,35.832459,128.547577,대구광역시 달서구
2,달비공원 공영주차장,노외,대구광역시 달서구 상화로 420,대구광역시 달서구 상인동 1594,35.807200,128.559447,대구광역시 달서구
3,죽전동제2공영주차장,노외,대구광역시 달서구 용산서로 10,대구광역시 달서구 용산동 931-5,35.857789,128.529562,대구광역시 달서구
4,달서시장 공영주차장,노외,대구광역시 달서구 당산로 37-14,대구광역시 달서구 본리동 110-5,35.841014,128.543167,대구광역시 달서구
...,...,...,...,...,...,...,...
15310,북샛말공영주차장,노외,경기도 이천시 이섭대천로 1299,경기도 이천시 창전동 443-25,37.286151,127.448467,경기도 이천시
15311,중리천로공영주차장,노외,경기도 이천시 중리천로 38,경기도 이천시 중리동 180-2,37.280280,127.442221,경기도 이천시
15312,설봉공원주차장,노외,경기도 이천시 경충대로2709번길 128,경기도 이천시 관고동 356,37.279460,127.427557,경기도 이천시
15313,백사 산수유꽃축제 주차장,노외,경기도 이천시 백사면 원적로775번길 17,경기도 이천시 백사면 도립리 1006-1,37.340781,127.462293,경기도 이천시


### null 확인

In [9]:
df2.isnull().sum()

주차장명           0
주차장유형          0
소재지도로명주소    5961
소재지지번주소      915
위도          2278
경도          2281
제공기관명          0
dtype: int64

* 시 / 구 / 동 정보 컬럼을 구분해야하고
* 위도 경도에 있는 null값을 채워야함

## 대전지역 주차장만 추출

In [10]:
df2 = df2[df2['제공기관명'].str.contains('대전', na=False)]
df2

,주차장명,주차장유형,소재지도로명주소,소재지지번주소,위도,경도,제공기관명
20,탄방동탄방역주변,노상,대전광역시 서구 계룡로553번길38,대전광역시 서구 탄방동667,NaN,NaN,대전광역시
123,유성IC만남의광장,노상,대전광역시 유성구 월드컵대로 31 (장대동),대전광역시 유성구 장대동 240-43,NaN,NaN,대전광역시
200,세천7 공한지 주차장,노외,NaN,대전광역시 동구 세천동 47-11,NaN,NaN,대전광역시
201,세천8 공한지 주차장,노외,NaN,대전광역시 동구 세천동 76-1,NaN,NaN,대전광역시
202,신도시장 주차장,노외,대전광역시 동구 비래서로42번길,대전광역시 동구 가양2동 39-6,NaN,NaN,대전광역시
...,...,...,...,...,...,...,...
15107,문화동제7노외,노외,대전광역시 중구 과례로 83,대전광역시 중구 문화동 13-9,NaN,NaN,대전광역시
15108,부사동제1노외,노외,대전광역시 중구 부용로 55,대전광역시 중구 부사동 423-6,NaN,NaN,대전광역시
15118,로히트~가을햇살,노상,대전광역시 유성구 농대로2번길 9,대전광역시 유성구 어은동 104-17~104-9,NaN,NaN,대전광역시
15119,문화2동제2노상,노상,대전광역시 중구 당디로36번길 113,대전광역시 중구 문화동 671-10,NaN,NaN,대전광역시


In [11]:
df2['제공기관명'].unique()

array(['대전광역시', '대전광역시 중구', '대전광역시 유성구', '대전광역시 대덕구', '대전광역시 서구',
       '대전광역시 동구'], dtype=object)

## 시, 구, 법정동 추출

대전광역시, **구, **동 형태인 `소재지지번주소`를 split

In [12]:
df2['시'] = '대전광역시'
df2['구'] = np.nan
df2['법정동'] = np.nan
df2['행정동'] = np.nan

### 시 정보 체크

In [13]:
# 모든 값이 대전광역시 형태인가?
df2['소재지지번주소'].str[:6].unique()

array(['대전광역시 ', nan, '봉명동 64'], dtype=object)

대전광역시, nan, 봉명동 3가지 형태

### 지번주소 null 체크

In [14]:
df2_null = df2[df2['소재지지번주소'].isnull()]
df_addr = df2[~df2['소재지지번주소'].isnull()]
df2_null

,주차장명,주차장유형,소재지도로명주소,소재지지번주소,위도,경도,제공기관명,시,구,법정동,행정동
2227,어은로58번길(카이스트옆),노상,대전광역시 유성구 어은로58번길,NaN,36.362589,127.356745,대전광역시 유성구,대전광역시,NaN,NaN,NaN
11222,이현동주차장,노외,대전광역시 대덕구 대청호수로1326번길 75,NaN,36.413739,127.466015,대전광역시 대덕구,대전광역시,NaN,NaN,NaN


도로명주소로 존재하여 우선 도로명주소로 채워줌

In [15]:
# 도로명주소로 채워주기
df2['소재지지번주소'].loc[2227] = df2['소재지도로명주소'].loc[2227]
df2['소재지지번주소'].loc[11222] = df2['소재지도로명주소'].loc[11222]

In [16]:
df2[df2['소재지지번주소'].isnull()]

,주차장명,주차장유형,소재지도로명주소,소재지지번주소,위도,경도,제공기관명,시,구,법정동,행정동


### 봉명동 체크

In [17]:
df_addr[df_addr['소재지지번주소'].str.contains('봉명동 64', na=False)] 

,주차장명,주차장유형,소재지도로명주소,소재지지번주소,위도,경도,제공기관명,시,구,법정동,행정동
2800,봉명동 카페거리,노상,NaN,봉명동 648,36.423183,127.393682,대전광역시 유성구,대전광역시,NaN,NaN,NaN


좌표로 검색해본 결과 관평동1공영주차장으로 나옴   
`대전 유성구 관평동 1175`

In [18]:
df2['소재지지번주소'].loc[2800] = '대전광역시 유성구 관평동 1175'
df2['소재지지번주소'].loc[2800]

'대전광역시 유성구 관평동 1175'

In [19]:
df2['소재지지번주소'].str[:6].unique()

array(['대전광역시 '], dtype=object)

### 구, 동 분리

In [20]:
df_split = df2.copy()

In [21]:
# 대전광역시 **구 **동 형태에서 구 확인
df_split['소재지지번주소'].str[6:9].unique()

array(['서구 ', '유성구', '동구 ', '중구 ', '대덕구'], dtype=object)

In [22]:
# **동 형태 확인
df_split['소재지지번주소'].str[10:14].unique()

array(['방동66', '장대동 ', '천동 4', '천동 7', '양2동 ', '상동 1', '하동 3', '동 51',
       '동 31', '운 34', '전동 6', '전동 1', '전동 2', '동 44', '동 30', '동 3-',
       '동 85', '동93-', '암동 2', '동 9-', '동천', '전천', '천동 2', '성동 4', '천동 3',
       '천동 1', '흥동 4', '사동 1', '영동 7', '교동 1', '사동 2', '화동 3', '화동 5',
       '창동 1', '동 13', '평동 3', '성동 1', '두동 3', '교동 3', '동 17', '정동 3',
       '정동 4', '화동 8', '성동 2', '화동 1', '도동 1', '양동 4', '양동 7', '월동 5',
       '동 25', '동 1-', '동 대동', '도동 8', '산1길,', '용호동 ', '삼정동 ', '중리동 ',
       '비래동 ', '읍내동 ', '장동 6', '석봉동 ', '장동 7', '장동 3', '신대동 ', '문평동 ',
       '이현동 ', '신탄진동', '송촌동 ', '미호동 ', '법동 4', '대화동 ', '운동 3', '운동 5',
       '운동 4', '운동28', '양동 3', '아@ 1', '양동 1', '양동 9', '암2차주', '암동50',
       '암1동 ', '암2동 ', '암동17', '양1동 ', '성동 3', '도동 6', '원내동 ', '노은동 ',
       '반석동 ', '봉산동 ', '구암동 ', '봉명동 ', '궁동 4', '어은로5', '어은동 ', '전민동 ',
       '도룡동 ', '신성동 ', '덕명동 ', '죽동 6', '계산동 ', '지족동 ', '관평동 ', '용산동 ',
       '송강동 ', '죽동 5', '성동 솔', '전로13', '정동 7', '촌A 2', '안동 2', '흥동 1',
  

In [23]:
# 공백으로 소재지전체주소 분리
df_split['구'] = df_split.소재지지번주소.str.split(' ').str[1] 
df_split['법정동'] = df_split.소재지지번주소.str.split(' ').str[2]

In [24]:
# 구, 법정동 정보가 정상적으로 분리됐는지 확인
print(df_split['구'].unique(), df_split['구'].nunique())
print(df_split['법정동'].unique(), df_split['법정동'].nunique())

['서구' '유성구' '동구' '중구' '대덕구'] 5
['탄방동667' '장대동' '세천동' '가양2동' '신상동' '신하동' '원동' '용운' '용전동' '인동' '정동'
 '중동93-2' '천동' '판암동' '중동' '대동천' '대전천' '유천동' '대성동' '대흥동' '부사동' '안영동' '대사동'
 '석교동' '선화동' '문화동' '문창동' '목동' '태평동' '산성동' '용두동' '호동' '사정동' '홍도동' '가양동'
 '낭월동' '대동' '동산1길,' '용호동' '삼정동' '중리동' '비래동' '읍내동' '장동' '석봉동' '신대동' '문평동'
 '이현동' '신탄진동' '송촌동' '미호동' '법동' '대화동' '용운동' '용운동282-277번지선' '자양동' '동아@'
 '판암2차주공@' '판암동503-2~507-2번지선' '판암1동' '판암2동' '판암동174-1' '가양1동' '삼성동' '원내동'
 '노은동' '반석동' '봉산동' '구암동' '봉명동' '궁동' '어은로58번길' '어은동' '전민동' '도룡동' '신성동'
 '덕명동' '죽동' '계산동' '지족동' '관평동' '용산동' '송강동' '태전로137번길(삼성동' '효촌A' '신안동' '신흥동'
 '충무로길(구' '대동복개천(새들뫼@' '동서대로104번길,' '교촌동' '세동' '복용동' '학하동' '화암동'
 '대흥동452-96' '월평동510' '월평동281' '월평1동29-3' '석교동12-10,' '오류동' '은행동' '중촌동'
 '옥계동' '대청호수로1326번길' '가양2동사무소' '갈마동274-7' '둔산동' '둔산동1306' '월평동663'
 '갈마동1435' '둔산동1136' '둔산동1001' '월평동160-20' '침산동' '만년동340' '둔산동1390'
 '탄방동746' '둔산동1300' '둔산2동1306' '둔산동957' '둔산2동' '월평동' '월평동29-3' '탄방동' '성남동'
 '소제동' '선화동136-11'] 128


In [25]:
df_split['법정동'] = np.nan

구 분리는 성공적으로 됐으나  
법정동 분리는 행정동과 섞여있고 주소 형태의 잡음으로 인해 실패

## kakao api 호출

In [26]:
df_kakao = df_split.copy()
df_kakao.reset_index(drop=True, inplace=True)

In [27]:
# rest키 발급 https://developers.kakao.com/console/app

# 병용 474aa3bdba9bd34f1560a7812fafeb9b
# 재인 08d64ac0652c8f9e1c4ea45118910c53
# 수영 616734aa8214dbf958cfaebd6bd41aec
# 태희 04e7e5d98a2c026956de6095963f28f8

rest_api_key = "474aa3bdba9bd34f1560a7812fafeb9b"

In [28]:
class KakaoLocalAPI:
    """
    Kakao Local API Controller
    """
    def __init__(self, rest_api_key):
        """
        Rest API key 초기화 및 기능 별 url 설정
        """
        
        # REST API key 설정
        self.rest_api_key = rest_api_key
        self.headers = {"Authorization": "KakaoAK {}".format(rest_api_key)}
        
        # 01 주소검색
        self.URL_01 = "https://dapi.kakao.com/v2/local/search/address.json"
        
    def search_address(self, query, analyze_type = None, page = None, size = None):
        """
        01 주소 검색
        """
        params = {"query": f"{query}"}
    
        if analyze_type != None:
            params["analyze_type"]=f"{analyze_type}"
    
        elif page != None:
            params['page'] = f"{page}"
    
        elif size != None:
            params['size'] = f"{size}"
    
        res = requests.get(self.URL_01, headers = self.headers, params = params)
        document = json.loads(res.text)
    
        return document

In [29]:
kakao = KakaoLocalAPI(rest_api_key)

In [30]:
add_list = df_kakao['소재지지번주소']
add_list[0]

'대전광역시 서구 탄방동667'

In [31]:
# api 호출 확인
kakao.search_address(add_list[0]) 

{'documents': [{'address': {'address_name': '대전 서구 탄방동 667',
    'b_code': '3017010600',
    'h_code': '3017055500',
    'main_address_no': '667',
    'mountain_yn': 'N',
    'region_1depth_name': '대전',
    'region_2depth_name': '서구',
    'region_3depth_h_name': '탄방동',
    'region_3depth_name': '탄방동',
    'sub_address_no': '',
    'x': '127.385564409802',
    'y': '36.344615015252'},
   'address_name': '대전 서구 탄방동 667',
   'address_type': 'REGION_ADDR',
   'road_address': {'address_name': '대전 서구 계룡로553번길 38',
    'building_name': '',
    'main_building_no': '38',
    'region_1depth_name': '대전',
    'region_2depth_name': '서구',
    'region_3depth_name': '탄방동',
    'road_name': '계룡로553번길',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '127.385566564485',
    'y': '36.3446227584109',
    'zone_no': '35262'},
   'x': '127.385564409802',
   'y': '36.344615015252'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [32]:
for i in tqdm(range(len(add_list))):
    try :
        df_kakao['경도'].iloc[i] = kakao.search_address(add_list[i])['documents'][0]['x']
        df_kakao['위도'].iloc[i] = kakao.search_address(add_list[i])['documents'][0]['y']
    except:
        pass
    
    try:
        df_kakao['법정동'].iloc[i] = kakao.search_address(add_list[i])['documents'][0]['address']['region_3depth_name']
    except:
        pass
    
    try:                
        df_kakao['행정동'].iloc[i] = kakao.search_address(add_list[i])['documents'][0]['address']['region_3depth_h_name']
    except:
        pass        
        
print(df_kakao.isnull().sum())

100%|██████████| 866/866 [03:38<00:00,  3.97it/s]

주차장명          0
주차장유형         0
소재지도로명주소    163
소재지지번주소       0
위도           69
경도           69
제공기관명         0
시             0
구             0
법정동          78
행정동          78
dtype: int64


In [33]:
df_1st_fail = df_kakao[df_kakao['위도'].isnull()]
df_1st_save = df_kakao[~df_kakao['위도'].isnull()]

print(df_1st_fail.shape)
print(df_1st_save.shape)

(69, 11)
(797, 11)


1차 추출 797개 성공  
69개 실패

## 2차 추출

In [34]:
df_1st_fail.head(10)

,주차장명,주차장유형,소재지도로명주소,소재지지번주소,위도,경도,제공기관명,시,구,법정동,행정동
20,한약·인쇄거리 노외공영주차장,노외,대전광역시 동구 태전로 27,"대전광역시 동구 중동 9-5, 10-16",NaN,NaN,대전광역시,대전광역시,동구,NaN,NaN
21,대신동 하상주차장,노외,"대전광역시 동구 대동천좌안, 대동천우안",대전광역시 동구 대동천,NaN,NaN,대전광역시,대전광역시,동구,NaN,NaN
22,선화교 하상주차장(삼성동),노외,대전광역시 동구 대전천동로,대전광역시 동구 대전천,NaN,NaN,대전광역시,대전광역시,동구,NaN,NaN
23,소제동 하상주차장,노외,"대전광역시 동구 대동천좌안, 대동천우안",대전광역시 동구 대동천,NaN,NaN,대전광역시,대전광역시,동구,NaN,NaN
60,"홍도빌라 주변(흑정골길, 3길, 6길)",노상,대전광역시 동구 옛신탄진로 55번길,대전광역시 동구 홍도동 175-15~139-11번지선,NaN,NaN,대전광역시,대전광역시,동구,NaN,NaN
66,대동이스트시티옆 공영주차장,노외,NaN,대전광역시 동구 대동 대동 107-37,NaN,NaN,대전광역시,대전광역시,동구,NaN,NaN
67,"홍도동 평화로운@, 대호빌라 주변",노상,대전광역시 동구 홍도로 33번길,대전광역시 동구 홍도동 89-92번지선,NaN,NaN,대전광역시,대전광역시,동구,NaN,NaN
68,홍도동 시영(아) 1동 옆,노상,대전광역시 동구 대전로1002번길,대전광역시 동구 홍도동 137-3~137-3번지선,NaN,NaN,대전광역시,대전광역시,동구,NaN,NaN
70,홍도동 시영아파트 주변,노상,"대전광역시 동구 동산1길, 흑정골길, 동산5길","대전광역시 동구 동산1길, 흑정골길, 동산5길",NaN,NaN,대전광역시,대전광역시,동구,NaN,NaN
99,신흥동 제1치수교 하상주차장,노외,대전광역시 동구 대동천우안2길,대전광역시 동구 대동천,NaN,NaN,대전광역시,대전광역시,동구,NaN,NaN


위도 경도가 없으면 행정동도 추출이 안됐음  
도로명주소에서 null 존재

In [35]:
df_road = df_1st_fail.copy()

### 도로명주소 null 값을 직접 검색

In [36]:
df_road[df_road['소재지도로명주소'].isnull()]

,주차장명,주차장유형,소재지도로명주소,소재지지번주소,위도,경도,제공기관명,시,구,법정동,행정동
66,대동이스트시티옆 공영주차장,노외,NaN,대전광역시 동구 대동 대동 107-37,NaN,NaN,대전광역시,대전광역시,동구,NaN,NaN
222,가양동 가양천 복개길,노상,NaN,대전광역시 동구 가양동 아침마을@~가양초교 맞은편,NaN,NaN,대전광역시,대전광역시,동구,NaN,NaN
223,가양동 가양천 복개길,노상,NaN,대전광역시 동구 가양동 대전한일신협~영동올갱이해장국,NaN,NaN,대전광역시,대전광역시,동구,NaN,NaN


* 66 : 주차장명과 소재지지번주소 모두 검색안됨
* 222 : 주차장명으로 매칭되는 건은 없으나 지도에서 추측되는 주차장 매칭 -> `대전 동구 가양동 333`
* 223 : 주차장명과 소재지지번주소 모두 매칭안됨

In [37]:
df_road['소재지도로명주소'][222] = '대전 동구 가양동 333'

In [38]:
df_road.dropna(axis=0, subset=['소재지도로명주소'],inplace=True)
df_road.reset_index(drop=True, inplace=True)
df_road.isnull().sum()

주차장명         0
주차장유형        0
소재지도로명주소     0
소재지지번주소      0
위도          67
경도          67
제공기관명        0
시            0
구            0
법정동         67
행정동         67
dtype: int64

### kakao api 호출 - 도로명주소

In [39]:
road_list = df_road['소재지도로명주소']
road_list[0]

'대전광역시 동구 태전로 27'

In [40]:
for i in tqdm(range(len(road_list))):
    try :
        df_road['경도'].iloc[i] = kakao.search_address(road_list[i])['documents'][0]['x']
        df_road['위도'].iloc[i] = kakao.search_address(road_list[i])['documents'][0]['y']
    except:
        pass
    
    try:
        df_road['법정동'].iloc[i] = kakao.search_address(road_list[i])['documents'][0]['address']['region_3depth_name']
    except:
        pass
    
    try:                
        df_road['행정동'].iloc[i] = kakao.search_address(road_list[i])['documents'][0]['address']['region_3depth_h_name']
    except:
        pass        
        
print(df_road.isnull().sum())

100%|██████████| 67/67 [00:16<00:00,  4.09it/s]

주차장명         0
주차장유형        0
소재지도로명주소     0
소재지지번주소      0
위도          14
경도          14
제공기관명        0
시            0
구            0
법정동         45
행정동         45
dtype: int64


In [41]:
df_2nd_fail = df_road[df_road['위도'].isnull()]
df_2nd_save = df_road[~df_road['위도'].isnull()]

print(df_2nd_fail.shape)
print(df_2nd_save.shape)

(14, 11)
(53, 11)


2차추출로 53개 추가 성공

In [42]:
# 병합
df_save= pd.concat([df_2nd_save, df_1st_save], 
                    axis=0,
                    join='outer'
                    )


df_save.reset_index(drop=True, inplace=True)

## 3차 추출 - 위도 경도로 동정보 추출


In [43]:
df_save.head(5)

,주차장명,주차장유형,소재지도로명주소,소재지지번주소,위도,경도,제공기관명,시,구,법정동,행정동
0,한약·인쇄거리 노외공영주차장,노외,대전광역시 동구 태전로 27,"대전광역시 동구 중동 9-5, 10-16",36.3329045197213,127.429433613512,대전광역시,대전광역시,동구,중동,중앙동
1,선화교 하상주차장(삼성동),노외,대전광역시 동구 대전천동로,대전광역시 동구 대전천,36.3144903620616,127.439903707812,대전광역시,대전광역시,동구,NaN,NaN
2,"홍도빌라 주변(흑정골길, 3길, 6길)",노상,대전광역시 동구 옛신탄진로 55번길,대전광역시 동구 홍도동 175-15~139-11번지선,36.3450589963811,127.422438852516,대전광역시,대전광역시,동구,NaN,NaN
3,"홍도동 평화로운@, 대호빌라 주변",노상,대전광역시 동구 홍도로 33번길,대전광역시 동구 홍도동 89-92번지선,36.3489720965201,127.427324330719,대전광역시,대전광역시,동구,NaN,NaN
4,홍도동 시영(아) 1동 옆,노상,대전광역시 동구 대전로1002번길,대전광역시 동구 홍도동 137-3~137-3번지선,36.3460997869619,127.42046438782,대전광역시,대전광역시,동구,NaN,NaN


In [44]:
# 위경도를 입력받는 kakao api 함수
def lat_lon_to_addr(lon,lat):
    url = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x={longitude}&y={latitude}'.format(longitude=lon,latitude=lat)
    headers = {"Authorization": "KakaoAK " + rest_api_key}
    result = json.loads(str(requests.get(url, headers=headers).text))
    match_first = result
    return match_first

In [45]:
df_lati = df_save['위도']
df_long = df_save['경도']

In [46]:
long_list = df_long
lati_list = df_lati

In [47]:
long_list[4], lati_list[4]

('127.42046438782', '36.3460997869619')

In [48]:
# 함수 확인
lat_lon_to_addr(long_list[4], lati_list[4])['documents'][0]['region_3depth_name']

'홍도동'

In [49]:
for i in tqdm(range(len(long_list))):
    try:
        df_save['법정동'].iloc[i] = lat_lon_to_addr(long_list[i], lati_list[i])['documents'][0]['region_3depth_name']
    except:
        pass   
    
    try :
        df_save['행정동'].iloc[i] = lat_lon_to_addr(long_list[i], lati_list[i])['documents'][1]['region_3depth_name']
    except:
        pass    

print(df_save.isnull().sum())

100%|██████████| 850/850 [01:37<00:00,  8.68it/s]

주차장명          0
주차장유형         0
소재지도로명주소    160
소재지지번주소       0
위도            0
경도            0
제공기관명         0
시             0
구             0
법정동           0
행정동           0
dtype: int64


In [50]:
df_save.head()

,주차장명,주차장유형,소재지도로명주소,소재지지번주소,위도,경도,제공기관명,시,구,법정동,행정동
0,한약·인쇄거리 노외공영주차장,노외,대전광역시 동구 태전로 27,"대전광역시 동구 중동 9-5, 10-16",36.3329045197213,127.429433613512,대전광역시,대전광역시,동구,중동,중앙동
1,선화교 하상주차장(삼성동),노외,대전광역시 동구 대전천동로,대전광역시 동구 대전천,36.3144903620616,127.439903707812,대전광역시,대전광역시,동구,효동,효동
2,"홍도빌라 주변(흑정골길, 3길, 6길)",노상,대전광역시 동구 옛신탄진로 55번길,대전광역시 동구 홍도동 175-15~139-11번지선,36.3450589963811,127.422438852516,대전광역시,대전광역시,동구,홍도동,홍도동
3,"홍도동 평화로운@, 대호빌라 주변",노상,대전광역시 동구 홍도로 33번길,대전광역시 동구 홍도동 89-92번지선,36.3489720965201,127.427324330719,대전광역시,대전광역시,동구,홍도동,홍도동
4,홍도동 시영(아) 1동 옆,노상,대전광역시 동구 대전로1002번길,대전광역시 동구 홍도동 137-3~137-3번지선,36.3460997869619,127.42046438782,대전광역시,대전광역시,동구,홍도동,홍도동


### 문제점 살펴보기

In [51]:
df_name = df_2nd_fail.copy()
df_name.reset_index(drop=True, inplace=True)

In [52]:
df_name

,주차장명,주차장유형,소재지도로명주소,소재지지번주소,위도,경도,제공기관명,시,구,법정동,행정동
0,대신동 하상주차장,노외,"대전광역시 동구 대동천좌안, 대동천우안",대전광역시 동구 대동천,NaN,NaN,대전광역시,대전광역시,동구,NaN,NaN
1,소제동 하상주차장,노외,"대전광역시 동구 대동천좌안, 대동천우안",대전광역시 동구 대동천,NaN,NaN,대전광역시,대전광역시,동구,NaN,NaN
2,홍도동 시영아파트 주변,노상,"대전광역시 동구 동산1길, 흑정골길, 동산5길","대전광역시 동구 동산1길, 흑정골길, 동산5길",NaN,NaN,대전광역시,대전광역시,동구,NaN,NaN
3,자양동 우송대학교 동 캠퍼스 담길,노상,대전광역시 동구 백룡길57번길,대전광역시 동구 자양동 98-17~97-3번지선,NaN,NaN,대전광역시,대전광역시,동구,NaN,NaN
4,삼성동 삼성물류센터 주변,노상,대전광역시 동구 현암시장5길,대전광역시 동구 삼성동 395번지 주변,NaN,NaN,대전광역시,대전광역시,동구,NaN,NaN
5,"삼성동 한국아파트 주변(현암로18번길, 태전로174번 안길)",노상,"대전광역시 동구 보건소 4,5길","대전광역시 동구 삼성동 373, 382,383번지 주변",NaN,NaN,대전광역시,대전광역시,동구,NaN,NaN
6,신흥삼거리~제1치수교,노상,대전광역시 동구 충무로길,대전광역시 동구 충무로길(구 신흥동사무소 앞),NaN,NaN,대전광역시,대전광역시,동구,NaN,NaN
7,용운동 도리어린이공원 주변,노상,대전광역시 동구 새울로23번길,대전광역시 동구 용운동 284번지 주변,NaN,NaN,대전광역시,대전광역시,동구,NaN,NaN
8,대동복개도로(대동시장길),노상,대전광역시 동구 대동복개천,대전광역시 동구 대동복개천(새들뫼@ 앞길),NaN,NaN,대전광역시,대전광역시,동구,NaN,NaN
9,대동복개도로(대동시장길),노상,대전광역시 동구 대동복개천,대전광역시 동구 대동복개천(새들뫼@ 앞길),NaN,NaN,대전광역시,대전광역시,동구,NaN,NaN


대체로 일정 범위에 걸쳐져있는 주소지인 경우가 많음 ex) 89-92번지선 등  
14개를 따로 카카오맵에서 수동 검색하여 근사한 주소지로 대체하려 했으나  
임의로 선택한 주차장 주소지가 기존에 추출한 주차장과 겹치는 이슈 발생하여 drop하기로 결정함

## 데이터프레임으로 저장

In [53]:
df_save.shape

(850, 11)

In [54]:
df_save.to_csv(PATH + "/06_주차장/주차장_장병용_전처리_행정동 추출(850).csv", index=False, encoding='cp949')